# Intelligent Systems 2023: 3rd practical assignment
## Informed Search

Your name: Chantal Elena Ariu

Your VUNetID: car103

If you do not provide your name and VUNetID we will not accept your submission.

### Learning objectives
At the end of this exercise you should be able to understand the implementations of the basic algorithms for informed search. You should be able to:

1. Understand the algorithms (be able to explain in your own words)
2. Follow the individual steps of the algorithms
3. Make small modifications of the code to see the effect on the search algorithms
4. Make small adaptations to the algorithm to study the computational properties

### Practicalities

Follow this Notebook step-by-step.

Of course, you can do the exercises in any Programming Editor of your liking.
But you do not have to. Feel free to simply write code in the Notebook. Please use your studentID+Assignment2.ipynb as the name of the
Notebook.  

Note: unlike the courses dedicated to programming we will not evaluate the style of the programs. But we will, however, test your programs on other data that we provide, and your program should give the correct output to the test-data as well.

As was mentioned, the assignment is graded as pass/fail. To pass you need to have either a full working code or an explanation of what you tried and what didn't work for the tasks that you were unable to complete (you can use multi-line comments or a text cell).

## Initialising

First, let us make sure the necessary packages are installed, and imported. Run the following code:

In [1]:
import sys
!{sys.executable} -m pip install numpy
import datetime
import functools
import heapq
import numpy as np
from numpy import random
from collections import deque

from utils import *

# This might produce a warning that numpy is already installed.


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


## Starting your first game

In [31]:
HAND_SIZE = 5      #TODO: replace with your desired hand size (3, 5 or 10)
STUDENT_NUMBER = 2824004 #TODO: replace with your own student number

With the constant HAND_SIZE we decide how many cards we want in our hand.
By default it is set to 5, you can change it to any hand size,
do keep in mind that the higher the number of cards in your hand,
the more branches in the search tree there are, and the longer it will take to run.

Your student number is used to set a random seed.
There are situations imaginable where you want a pseudo random selection (for example when debugging it's nice to always work with the same values)
In short, the seed ensures that you get a pseudo random distribution that will always be the same when you re-run the code.
It is a random distribution because you don't have to hard code them in yourself,
but it is not random in the sense that the next time you run it you get different cards!
For more information on pseudo random number generators,
check out https://www.geeksforgeeks.org/pseudo-random-number-generator-prng/.

### Updated representation

You may wonder how the cards for this game are represented.
Go to utils and find out in which variable this information is found, print this variable below:

In [32]:
print(representation)


        h  d  s  c
ace  [[ 0  1  2  3]  = 11pts     h = hearts
ten   [ 4  5  6  7]  = 10pts     d = diamonds
king  [ 8  9 10 11]  = 4pts      s = spades
queen [12 13 14 15]  = 3pts      c = clubs
jack  [16 17 18 19]] = 2pts
For example: '10' is 'king of spades'



Note that cards have values now w.r.t. the game from Assignment 2

### Some support functions

Next, there are some functions we need for the implementation. Try to get the gist of what they do, but if you do not understand fully, don't worry. The first one returns the points value of a card, the second tests whether a move is valid (so whether a card follows suit, or has the same value). The third is a helper function that checks whether two cards have the same suit, and the fourth function checks whether two cards have the same value. The last one makes a random choice of cards in the hands.

We don't expect you to fully understand the coding behind these functions, however, if you are interested you might find the following link useful: https://www.programiz.com/python-programming/matrix. Take a look at how Python Matrices are created and how to access rows and columns.

In [33]:
def get_points(cardA):
    g = np.arange(20).reshape(5, 4) #this produces the same grid as the representation
    if (np.any(g[0] == cardA)): #Ace (11)
        return 11
    elif (np.any(g[1] == cardA)): #Ten (10)
        return 10
    elif (np.any(g[2] == cardA)): #King (4)
        return 4
    elif (np.any(g[3] == cardA)): #Queen (3)
        return 3
    elif (np.any(g[4] == cardA)): #Jack (2)
        return 2

def valid_move(cardA, cardB):
    #print("validMove: comparing " + str(cardA) + " to " + str(cardB)) UNCOMMENT THIS TO SEE WHICH CARDS ARE BEING COMPARED
    g = np.arange(20).reshape(5, 4) #this produces the same grid as the representation, for the purpose of checking moves
    if check_value(cardA, cardB, g):
        return True
    elif check_suit(cardA, cardB, g):
        return True
    else:
        #print("validMove: No move found")
        return False

def check_suit(cardA, cardB, grid):
    r, c = grid.shape
    for i in range(c):
        if np.any(grid[:, i] == cardA) and np.any(grid[:, i] == cardB):
            return True

def check_value(cardA, cardB, grid):
    r, c = grid.shape
    for i in range(r):
        if np.any(grid[i] == cardA) and np.any(grid[i] == cardB):
            return True

def pick_cards(seed, size):
    random.seed(seed)
    cards = np.random.choice(20, (size*2), replace = False)
    leftHand = cards[:size]
    rightHand = cards[size:]
    return (leftHand, rightHand)

### Breadth, and Depth First Search

From assignment 1 we are already familiar with the Breadth First and Depth First search algorithms. See the code below:

In [34]:
def breadth_first_tree_search(problem):
    """
    Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument fringe should be an empty queue.
    Repeats infinitely in case of loops.
    """
    counter = 0
    fringe = deque([Node(problem.initial)])  # FIFO queue
    while fringe:
        node = fringe.popleft()
        counter += 1
        if problem.goal_test(node.state):
            print("###########")
            print("success!")
            print("solution: {}".format(node.solution()))
            print("{} Nodes generated".format(counter))
            return node
        fringe.extend(node.expand(problem))
    print("###########")
    print("unfortunately no solution has been found!")
    print("{} Nodes generated".format(counter))
    return None

In [35]:
def depth_first_tree_search(problem):
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument fringe should be an empty queue.
    Repeats infinitely in case of loops.
    """
    counter = 0
    fringe = [Node(problem.initial)]  # Stack
    while fringe:
        node = fringe.pop()
        counter += 1
        if problem.goal_test(node.state):
            print("###########")
            print("succes!")
            print("solution: {}".format(node.solution()))
            print("{} Nodes generated".format(counter))
            return node
        fringe.extend(node.expand(problem))
    print("###########")
    print("unfortunately no solution has been found!")
    print("{} Nodes generated".format(counter))
    return None

### Preparing the game
To run a search, we need to define an initial state. Run the cells below to generate hands automatically, or define a custom set of hands. Please note that if you use a custom set of hands, you need to replace 'leftHand' and 'rightHand' with 'customLeftHand' and 'customRightHand' at gamestate initialisation.

The cell below generates the cards for your left and right hand.

In [36]:
# leftHand, rightHand = pick_cards(2824004, 5)
# print(leftHand, rightHand)
customLeftHand = [16, 9, 5, 18, 17]
customRightHand = [8, 1, 14, 19, 11]

print("Left hand: {}".format(customLeftHand)) #or customLeftHand
print("Right hand: {}".format(customRightHand)) #or customRightHand

Left hand: [16, 9, 5, 18, 17]
Right hand: [8, 1, 14, 19, 11]


Now we define an initial state, and the problem.

In [37]:
initialState = GameState(customLeftHand, customRightHand, True, True)
initialState.printState()

p = Problem(initialState)
print("There are {} points needed to win(reach goal state)".format(p.winPoints))
print("There are {} points in the game".format(sum(p.initial.allCardPoints)))


------
GameState: Printing state: 
Left hand: [16, 9, 5, 18, 17]
Right hand: [8, 1, 14, 19, 11]
Do we play from left hand to get to next state? True
Points scored: 0
All cards: [16, 9, 5, 18, 17, 8, 1, 14, 19, 11]
All card points: [2, 4, 10, 2, 2, 4, 11, 3, 2, 4]
------
There are 33 points needed to win(reach goal state)
There are 44 points in the game


## Task 1:

Run Breadth and Depth first search and implement a counter in the algorithm above that prints the number of nodes generated (as you did in the previous assignment, although now to print you must uncomment the print statements instead of formulating your own).

In [38]:
startTime = datetime.datetime.now()
s=breadth_first_tree_search(p)
endTime = datetime.datetime.now()
duration = endTime - startTime
print("The Breadth First Search Took {} Microseconds".format(duration.microseconds))
print("BFS PATH_COST: {}".format(s.path_cost))

startTime = datetime.datetime.now()
s=depth_first_tree_search(p)
endTime = datetime.datetime.now()
duration = endTime - startTime
print("The Depth First Search Took {} Microseconds".format(duration.microseconds))
print("DFS PATH_COST: {}".format(s.path_cost))


###########
success!
solution: [5, 1, 9, 8, 16, 19]
69 Nodes generated
The Breadth First Search Took 22572 Microseconds
BFS PATH_COST: 149
###########
succes!
solution: [17, 19, 16, 8, 9, 1, 5]
11 Nodes generated
The Depth First Search Took 3091 Microseconds
DFS PATH_COST: 61


### Greedy Best First Search

See the code below for the Greedy Best First Search Algorithm

In [39]:
def greedy_best_first_search(problem, f, display=True):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; """
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    counter = 0
    while frontier:
        node = frontier.pop()
        counter += 1
        if problem.goal_test(node.state):
            if display:
                print("Search succesful!")
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
                print("solution: {}".format(node.solution()))
                print("{} Nodes generated".format(counter))
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    print("Search failed")
    print("{} Nodes generated".format(counter))
    return None

## Task 2

Run A Greedy Best First Search and implement a counter in the algorithm above that prints the number of nodes generated (as you did in the previous assignment, although now to print you must uncomment the print statements instead of formulating your own).

In [40]:
startTime = datetime.datetime.now()
greedy_best_first_search(p, p.h, True)
endTime = datetime.datetime.now()
duration = endTime - startTime
print("The Greedy Best First Search Took {} Microseconds".format(duration.microseconds))

Search succesful!
6 paths have been expanded and 6 paths remain in the frontier
solution: [5, 1, 9, 8, 16, 19]
7 Nodes generated
The Greedy Best First Search Took 4848 Microseconds


## Task 3

In what way do Depth First Search, Breadth First Search and Greedy Best First Search (using the distance to the goal as heuristic) differ?

Return a (brief) report to the multi line string below, take into account the number of nodes counted from the counter you implemented:


In [41]:
myReport1 = """
The most apparent difference is the amount of nodes generated in DFS, BFS and Greedy Best First Search. DFS generated 11 nodes, 
BFS generated 69 nodes and Greedy Best First Search generated 7 nodes. This means that Greedy Best First Search generated the
least amount of nodes, meaning it checked less nodes until it found the goal state compared to BFS and DFS. This is due to 
Greedy Best First Search using heuristics which allows for an informed search to take place compared to the uninformed searches
happening in DFS and BFS.
"""

## Task 4

Run a Greedy Best First Search (again) but this time print the path cost (uncomment the print statement and fill in the right variable.)

(Hint: Go into utils.py, in class Node you can see a variable that should help you. Remember that a -successful- search returns a Node which is saved as 'search' here)

In [42]:
search = greedy_best_first_search(p, p.h)

print("Path cost: {}".format(search.path_cost))

Search succesful!
6 paths have been expanded and 6 paths remain in the frontier
solution: [5, 1, 9, 8, 16, 19]
7 Nodes generated
Path cost: 149


### A* Search algorithm

See the code for the A star search below:

In [43]:
def astar_search(problem, h=None, display=True):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    return greedy_best_first_search(problem, lambda n: n.path_cost + h(n), display)

## Task 5

Consider the the solution of a greedy best first search and an a star search:

In [44]:
print("Greedy Best First Search: ")
gbf=greedy_best_first_search(p, p.h, True)
print("Path cost: {}".format(gbf.path_cost))

print("A Star Search: ")
ast=astar_search(p, p.h) # Needed to add the argument p.h otherwise the algorithm will try to call a function that wasn't passed
print("Path cost: {}".format(ast.path_cost))


Greedy Best First Search: 
Search succesful!
6 paths have been expanded and 6 paths remain in the frontier
solution: [5, 1, 9, 8, 16, 19]
7 Nodes generated
Path cost: 149
A Star Search: 
Search succesful!
26 paths have been expanded and 9 paths remain in the frontier
solution: [17, 19, 16, 8, 9, 1, 5]
27 Nodes generated
Path cost: 61


Explain in your own words what the difference between the two algorithms is, and why the solutions differ. Return your explanation to the multi line string below:

In [45]:
myReport2 = """
Since A star search combines the use of heuristics with the cost of reaching the goal from the current node it is at,
A star search is less expensive and more optimal since it will always find the least expensive way to get to the goal state,
whereas Greedy Best First Search takes the first path to the goal it can find, which in turn, leads to an unoptimal solution.
This can also be seen by the fact that Greedy Best First Search only explored 6 paths until it stops whereas A star explored 
26 paths, which is also the reason for why A star was able to find a path with the cost of only 61 compared to the path cost
of 149 that the path generated through Greedy Best First search.
"""

## Final Task: Collect all the results

Uncomment and run this cell (and all the cells above) to generate the text file that you have to hand in together with the notebook on canvas!

In [46]:
exportToText("assignment3.txt",STUDENT_NUMBER, customLeftHand, customRightHand, myReport1, myReport2 )